In [ ]:
# 9.9.21
# First let's plot the ecoblock graph

In [1]:
import importlib
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
import random
from operator import add
from graphviz import Source, render
import pydot
import csv  # jaimie do you need this?

import datetime
import time

import sys
sys.path.append('py_modules') # below modules are in this folder
#print(sys.path)
import setup_nx # your own module, setup.nx.py
importlib.reload(setup_nx)
import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm

In [7]:
# [Essential] specify input feeder data for IEEE 123-node test feeder

# Enter the path/name of the impedance model data (excel file)
filepath = "feeder_impedance_models/"
#modelpath = filepath + "ecoblock_impedexcel_trueZ2.xls"
modelpath = filepath + "OaklandJ_impedexcel_short.xlsx"

# load data is not actually used in assessment, but the feeder object builds variables to hold load info
loadfolder = filepath
loadpath = filepath + 'ecoblock_NL_PVpen100_sigbuilder.csv'
headerpath = filepath + 'ecoblock_time_header.csv'
load_data = loadpath

#file_name = string specifying name of dot file created when make_graph() is called
file_name = 'oaklandJ'
#file_name = 'ecoblock'

# Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 4160
Vbase = Vbase_ll / np.sqrt(3)
Sbase = 5000/3
substation_name = 'bus_nSOURCE' # formatted as 'bus_XXX'
timesteps = 1

# initialize some variables
ts = time.time()
print(datetime.datetime.fromtimestamp(ts))
plot = 0 #turn plot on/off
depths = {}
leaves = []

2021-12-29 13:38:45.083628


In [8]:
# [ESSENTIAL] create feeder object and plot feeder with make_graph func

fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix
R,X=hm.createRXmatrices_3ph(fin_feeder, node_index_map,depths)

#print('depths=',depths) # check this is populated, lists how far each node is from substation
#print('depths length=',len(depths))

# print list of first 10 buses in network
count = 0 
for i in fin_feeder.network:    
    print(i) 
    count += 1
    if count >= 10:
        break
    
graph = fin_feeder.network
print('node index map len=',len(node_index_map))

Finished initializing feeder
bus_n100
bus_n101
bus_n102
bus_n103
bus_n104
bus_n105
bus_n106
bus_n107
bus_n108
bus_n109
node index map len= 239


In [ ]:
# mark actuators on ecoblock graph
lst_act_locs=['bus_4','bus_12','bus_3079','bus_3089','bus_3011','bus_3015','bus_3025','bus_3029','bus_3033','bus_3037','bus_3038','bus_3044','bus_3049','bus_3056','bus_3058','bus_3057','bus_3059','bus_3063','bus_3062','bus_3001','bus_3008']
#[4 3079 3089 3011 3015 3025 3029 3033 3037 3038 3044 3049 3056 3058 3057 3059 3063 3062 3001 3008];
vis.markActuatorConfig(lst_act_locs, fin_feeder, file_name)

In [47]:
# gather impedance info Xii for each actuator
# read in sigbuilder csv. For each col (phase-act) up to half of all cols, remove take substring before / delimiter, append bus_, then add to cby_load

opened_csv_file = open('feeder_impedance_models/OaklandJ_sigbuilder.csv', 'r')
reader = csv.reader(opened_csv_file)

loads_table = []
for row in reader:
        loads_table.append(row)
#print(loads_table[0]) # header
lst_act_locs=[]
for phaseAct_name in loads_table[0]:
    str1 = phaseAct_name.split("/",1)[0] # LD_nXXX
    str2=str1.replace('LD_', 'bus_') # bus_nXXX
    lst_act_locs.append(str2) 
lst_act_locs.pop(0) # remove 'Time' from list
print(lst_act_locs[0:5])

# compute impedances to substation
x_acts=[] # separate  r and x so easier to create complex number in matlab
r_acts=[]
for cby_load in lst_act_locs:
    Zii=imp.get_total_impedance_from_substation(fin_feeder, cby_load, depths) # returns dict of 3x3 numpy array for Z_ii
    print('zii=',Zii[0,0])
    x_acts.append(np.imag(Zii[0,0]))
    r_acts.append(np.real(Zii[0,0]))
    #print('zii of ',cby_load,' = ',Zii)
print(x_acts[0:5])

# now write lst_act_locs to first row of csv, and Zii to second row of csv
with open('oaklandJ_zToSub.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(loads_table[0]) # same header as sigbuilder
    writer.writerow(r_acts)
    writer.writerow(x_acts)

['bus_n100', 'bus_n100', 'bus_n100', 'bus_n101', 'bus_n102']
zii= (1.4221403372794819+1.529430828516419j)
zii= (1.4221403372794819+1.529430828516419j)
zii= (1.4221403372794819+1.529430828516419j)
zii= (1.4251119675435406+1.5346498609887327j)
zii= (0.9166051040791535+1.2032803181361384j)
zii= (1.3622963527270469+1.50107496388084j)
zii= (1.3622963527270469+1.50107496388084j)
zii= (1.3622963527270469+1.50107496388084j)
zii= (0.7886019834265614+1.1190313456912748j)
zii= (0.7659395596157914+1.1115572365264517j)
zii= (0.7659395596157914+1.1115572365264517j)
zii= (0.7659395596157914+1.1115572365264517j)
zii= (1.3168705222839696+1.4795508506615065j)
zii= (1.3168705222839696+1.4795508506615065j)
zii= (1.3168705222839696+1.4795508506615065j)
zii= (0.7821493647697114+1.1294678221827095j)
zii= (0.7821493647697114+1.1294678221827095j)
zii= (0.7821493647697114+1.1294678221827095j)
zii= (1.3851825807860156+1.5317469479484567j)
zii= (1.3851825807860156+1.5317469479484567j)
zii= (1.3851825807860156+1.5

zii= (3.75586943e-05+3.75586943e-05j)
zii= (3.75586943e-05+3.75586943e-05j)
zii= (1.0548241221556154+1.084959669462042j)
zii= (1.0346092307143946+1.0739542364733063j)
zii= (0.2751067162636054+0.40722469297462294j)
zii= (0.2751067162636054+0.40722469297462294j)
zii= (0.2751067162636054+0.40722469297462294j)
zii= (0.2658130798350224+0.38598097600718306j)
zii= (0.2658130798350224+0.38598097600718306j)
zii= (0.2658130798350224+0.38598097600718306j)
zii= (0.2711704546150784+0.39822704937045567j)
zii= (0.2711704546150784+0.39822704937045567j)
zii= (0.2711704546150784+0.39822704937045567j)
zii= (0.27754425520377934+0.4127965041531742j)
zii= (0.27754425520377934+0.4127965041531742j)
zii= (0.27754425520377934+0.4127965041531742j)
zii= (0.2767679241809874+0.41102193973504025j)
zii= (0.2767679241809874+0.41102193973504025j)
zii= (0.2767679241809874+0.41102193973504025j)
zii= (0.06034273873681335+0.06789032267228898j)
zii= (0.06034273873681335+0.06789032267228898j)
zii= (0.06034273873681335+0.0678

zii= (0.5418060778840915+0.9075819222471401j)
zii= (0.5449482836462395+0.914096354608667j)
zii= (0.5449482836462395+0.914096354608667j)
zii= (0.5449482836462395+0.914096354608667j)
zii= (0.6360479556213452+0.9680370228483338j)
zii= (0.6360479556213452+0.9680370228483338j)
zii= (0.6360479556213452+0.9680370228483338j)
zii= (0.6657098626234923+1.0008111445015748j)
zii= (0.6657098626234923+1.0008111445015748j)
zii= (0.6657098626234923+1.0008111445015748j)
zii= (1.398881540207301+1.5601476802137615j)
zii= (1.398881540207301+1.5601476802137615j)
zii= (1.398881540207301+1.5601476802137615j)
zii= (0.6845995729623973+1.0216828188884224j)
zii= (0.6845995729623973+1.0216828188884224j)
zii= (0.6845995729623973+1.0216828188884224j)
zii= (1.5120082331335767+1.5720129177436204j)
zii= (1.5120082331335767+1.5720129177436204j)
zii= (1.5120082331335767+1.5720129177436204j)
zii= (1.4690337853160096+1.5516503427209634j)
zii= (1.4690337853160096+1.5516503427209634j)
zii= (1.4690337853160096+1.5516503427209

zii= (0.6075171108477043+0.9365126378041299j)
zii= (0.6075171108477043+0.9365126378041299j)
zii= (0.6075171108477043+0.9365126378041299j)
zii= (1.0209482846117124+1.018281499666389j)
zii= (0.5787404764352623+0.9047166741482545j)
zii= (0.5787404764352623+0.9047166741482545j)
zii= (0.5787404764352623+0.9047166741482545j)
zii= (1.0585939466631782+1.0387766288254392j)
zii= (0.6199803160336422+0.876244903641685j)
zii= (0.6199803160336422+0.876244903641685j)
zii= (0.6199803160336422+0.876244903641685j)
zii= (0.6089237016267792+0.8123052229623208j)
zii= (1.0792948758702663+1.0488383135576536j)
zii= (0.47493749263244345+0.7689497292251669j)
zii= (0.47493749263244345+0.7689497292251669j)
zii= (0.47493749263244345+0.7689497292251669j)
zii= (0.5550527877426296+0.8785436033864772j)
zii= (0.5550527877426296+0.8785436033864772j)
zii= (0.5550527877426296+0.8785436033864772j)
zii= (0.49095894892387143+0.8021654704095763j)
zii= (0.49095894892387143+0.8021654704095763j)
zii= (0.49095894892387143+0.80216

In [ ]:
# set config using lst_act_locs, and call Gdisc kgain design func. The func does:
# for 3ph system there are r phase-actuators, for example node6_phA; thus there are r nonzero F elements
# for each of nonzero element of F, det Gdisc conditions; conditions give an upper bound on that F ele search space 
# compute vector of F_ub, dimension rx1
# search space of [0 F_ub] in r-space and det evals of closed-loop system
# Select combo with smallest spectral radius, and return kgains of dim rx1


In [ ]:
# print kgains vector to csv, and in ephasorsim we read in csv to assign to Kp_vmag, Ki_vmag, etc.
# kgains has dimension rx1, where is number of phase-actuators)

In [5]:
print(node_index_map)

{'bus_n100': 0, 'bus_n101': 1, 'bus_n102': 2, 'bus_n103': 3, 'bus_n104': 4, 'bus_n105': 5, 'bus_n106': 6, 'bus_n107': 7, 'bus_n108': 8, 'bus_n109': 9, 'bus_n10': 10, 'bus_n110': 11, 'bus_n111': 12, 'bus_n112': 13, 'bus_n113': 14, 'bus_n114': 15, 'bus_n115': 16, 'bus_n116': 17, 'bus_n117': 18, 'bus_n118': 19, 'bus_n119': 20, 'bus_n11': 21, 'bus_n120': 22, 'bus_n121': 23, 'bus_n122': 24, 'bus_n123': 25, 'bus_n124': 26, 'bus_n125': 27, 'bus_n126': 28, 'bus_n127': 29, 'bus_n128': 30, 'bus_n129': 31, 'bus_n12': 32, 'bus_n130': 33, 'bus_n131': 34, 'bus_n132': 35, 'bus_n133': 36, 'bus_n134': 37, 'bus_n135': 38, 'bus_n136': 39, 'bus_n137': 40, 'bus_n138': 41, 'bus_n139': 42, 'bus_n13': 43, 'bus_n140': 44, 'bus_n141': 45, 'bus_n142': 46, 'bus_n143': 47, 'bus_n144': 48, 'bus_n145': 49, 'bus_n146': 50, 'bus_n147': 51, 'bus_n148': 52, 'bus_n149': 53, 'bus_n14': 54, 'bus_n150': 55, 'bus_n151': 56, 'bus_n152': 57, 'bus_n153': 58, 'bus_n154': 59, 'bus_n155': 60, 'bus_n156': 61, 'bus_n157': 62, 'bus_n

In [9]:
# ------------- do a basic eval_config ----------------------------
parmObj=hm.configParms()
# parmObj.set_ctrlTypes(['VVC', 'VVC', 'VWC', 'VWC', 'VWC', 'VVC', 'VVC', 'VVC'])
# parmObj.set_version(2) # 1 for PBC

parmObj.set_ctrlTypes(['PBC', 'PBC'])
parmObj.set_version(1) # 1 for PBC

#all_act_locs = ['bus_152', 'bus_21', 'bus_58', 'bus_70', 'bus_4', 'bus_82', 'bus_109', 'bus_47']
all_act_locs = ['bus_n103','bus_n156']
perf_nodes=['bus_n103','bus_n156']
# print(len(all_act_locs))
# print(len(foo))
feas, maxError,numfeas,bestF,indicMat=hm.eval_config(parmObj,fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)


[updateStateSpace] indicMat_table=
 [[  0   9   9]
 [  0 726 726]
 [  0  10  10]
 [  0 727 727]
 [  0  11  11]
 [  0 728 728]
 [  1 183 183]
 [  1 900 900]
 [  1 184 184]
 [  1 901 901]
 [  1 185 185]
 [  1 902 902]] << [bus indicMat_row indicMat_col], 3ph nodes should have 6 rows
num_fele= 12
working with  0 _th nonzero F ele
solving Gdisc conditions...
cond1= 3.00214992776168*f - 2
working with  1 _th nonzero F ele
solving Gdisc conditions...
cond1= 1.50107496388084*f - 2
working with  2 _th nonzero F ele
solving Gdisc conditions...
cond1= 3.00214992776168*f - 2
working with  3 _th nonzero F ele
solving Gdisc conditions...
cond1= 1.50107496388084*f - 2
working with  4 _th nonzero F ele
solving Gdisc conditions...
cond1= 3.00214992776168*f - 2
working with  5 _th nonzero F ele
solving Gdisc conditions...
cond1= 1.50107496388084*f - 2
working with  6 _th nonzero F ele
solving Gdisc conditions...
cond1= 3.14844474248726*f - 2
working with  7 _th nonzero F ele
solving Gdisc conditions...

F stable!
candFset= [0.08195324358061086, 0.1626833044212126, 0.08195324358061086, 0.1626833044212126, 0.08195324358061086, 0.1626833044212126, 0.0999039510971259, 0.19824690295835917, 0.0999039510971259, 0.19824690295835917, 0.0999039510971259, 0.19824690295835917]
% of fub= [0.12231827400091172, 0.12231827400091172, 0.12231827400091172, 0.12231827400091172, 0.12231827400091172, 0.12231827400091172, 0.1560999235892592, 0.1560999235892592, 0.1560999235892592, 0.1560999235892592, 0.1560999235892592, 0.1560999235892592]
F stable!
candFset= [0.07595370059820458, 0.1507737638740479, 0.07595370059820458, 0.1507737638740479, 0.07595370059820458, 0.1507737638740479, 0.10125087514656811, 0.20091970536897108, 0.10125087514656811, 0.20091970536897108, 0.10125087514656811, 0.20091970536897108]
% of fub= [0.11336373223612624, 0.11336373223612622, 0.11336373223612624, 0.11336373223612622, 0.11336373223612624, 0.11336373223612622, 0.15820449241651266, 0.15820449241651266, 0.15820449241651266, 0.1582

candFset= [0.17047561319592786, 0.33840681425460306, 0.17047561319592786, 0.33840681425460306, 0.17047561319592786, 0.33840681425460306, 0.18972621071164106, 0.37648794938091273, 0.18972621071164106, 0.37648794938091273, 0.18972621071164106, 0.37648794938091273]
% of fub= [0.25444121372526546, 0.25444121372526546, 0.25444121372526546, 0.25444121372526546, 0.25444121372526546, 0.25444121372526546, 0.29644720423693915, 0.29644720423693915, 0.29644720423693915, 0.29644720423693915, 0.29644720423693915, 0.29644720423693915]
candFset= [0.14468750958592239, 0.2872155041033982, 0.14468750958592239, 0.2872155041033982, 0.14468750958592239, 0.2872155041033982, 0.1378753092285461, 0.2735963167503962, 0.1378753092285461, 0.2735963167503962, 0.1378753092285461, 0.2735963167503962]
% of fub= [0.21595150684466027, 0.21595150684466027, 0.21595150684466027, 0.21595150684466027, 0.21595150684466027, 0.21595150684466027, 0.21543017066960327, 0.2154301706696033, 0.21543017066960327, 0.2154301706696033, 0

candFset= [0.2735705675676241, 0.5430579923357314, 0.2735705675676241, 0.5430579923357314, 0.2735705675676241, 0.5430579923357314, 0.221457387044907, 0.4394545024172373, 0.221457387044907, 0.4394545024172373, 0.221457387044907, 0.4394545024172373]
% of fub= [0.40831427995167774, 0.4083142799516777, 0.40831427995167774, 0.4083142799516777, 0.40831427995167774, 0.4083142799516777, 0.3460271672576672, 0.3460271672576672, 0.3460271672576672, 0.3460271672576672, 0.3460271672576672, 0.3460271672576672]
candFset= [0.28634136230646545, 0.568408972936715, 0.28634136230646545, 0.568408972936715, 0.28634136230646545, 0.568408972936715, 0.34443309240769643, 0.6834844177465226, 0.34443309240769643, 0.6834844177465226, 0.34443309240769643, 0.6834844177465226]
% of fub= [0.4273751676215902, 0.4273751676215902, 0.4273751676215902, 0.4273751676215902, 0.4273751676215902, 0.4273751676215902, 0.5381767068870257, 0.5381767068870257, 0.5381767068870257, 0.5381767068870257, 0.5381767068870257, 0.53817670688

candFset= [0.35336241784880595, 0.70145076975957, 0.35336241784880595, 0.70145076975957, 0.35336241784880595, 0.70145076975957, 0.4323270053907309, 0.8578989013222316, 0.4323270053907309, 0.8578989013222316, 0.4323270053907309, 0.8578989013222316]
% of fub= [0.527406593804188, 0.527406593804188, 0.527406593804188, 0.527406593804188, 0.527406593804188, 0.527406593804188, 0.675510945923017, 0.675510945923017, 0.675510945923017, 0.675510945923017, 0.675510945923017, 0.675510945923017]
candFset= [0.3208621576222587, 0.6369353278173195, 0.3208621576222587, 0.6369353278173195, 0.3208621576222587, 0.6369353278173195, 0.3398239411447739, 0.6743381332091607, 0.3398239411447739, 0.6743381332091607, 0.3398239411447739, 0.6743381332091607]
% of fub= [0.4788987427197891, 0.4788987427197891, 0.4788987427197891, 0.4788987427197891, 0.4788987427197891, 0.4788987427197891, 0.5309749080387092, 0.5309749080387092, 0.5309749080387092, 0.5309749080387092, 0.5309749080387092, 0.5309749080387092]
candFset= [